**<span style="color:red">THE NOTEBOOK IS WORK-IN-PROGRESS.</span>**

**<span style="color:red">Please upvote it if you find it interesting or usefull</span>**

# Exploratory Data Analysis of the data provided by Vesta Corporation

Here let's mention the dataset description from [this](https://www.kaggle.com/c/ieee-fraud-detection/discussion/101203) post:

## Transaction Table

* **TransactionDT:** timedelta from a given reference datetime (not an actual timestamp).
* **TransactionAMT:** transaction payment amount in USD.
* **ProductCD:** product code, the product for each transaction.
* **card1 - card6:** payment card information, such as card type, card category, issue bank, country, etc.
* **addr:** address.
* **dist:** distance.
* **P_emaildomain:** purchaser email domain.
* **R_emaildomain:** recipient email domain.
* **C1-C14:** counting, such as how many addresses are found to be associated with the payment card, etc. The actual meaning is masked.
* **D1-D15:** timedelta, such as days between previous transaction, etc.
* **M1-M9:** match, such as names on card and address, etc.
* **Vxxx:** Vesta engineered rich features, including ranking, counting, and other entity relations.

### Categorical Features:
* ProductCD.
* card1 - card6.
* addr1, addr2.
* Pemaildomain Remaildomain.
* M1 - M9.

## Identity Table (variables in this table are identity information):
* network connection information (IP, ISP, Proxy, etc).
* digital signature (UA/browser/os/version, etc) associated with transactions. 

They're collected by Vesta’s fraud protection system and digital security partners.

The field names are masked and pairwise dictionary will not be provided for privacy protection and contract agreement.

### Categorical Features:
* DeviceType.
* DeviceInfo.
* id12 - id38.

# All imports necessary

In [ ]:
import numpy as np

import pandas as pd

import os

import matplotlib_venn

import matplotlib.pyplot as plt

import seaborn as sns

import sklearn

from sklearn.naive_bayes import BernoulliNB

from sklearn.linear_model import LogisticRegression

from sklearn.svm import NuSVC

from sklearn.svm import SVC

from sklearn.model_selection import cross_validate

from sklearn.model_selection import StratifiedKFold

# A bit of configuration for pandas & visualizations

In [ ]:
pd.options.display.max_rows = 999

In [ ]:
sns.set(font_scale=2)

# Data location

In [ ]:
! ls -alh ../input

In [ ]:
! ls -alh ../input/ieee-fraud-detection/

In [ ]:
folder_path = '../input/ieee-fraud-detection/'

# Read the data

In [ ]:
train_identity = pd.read_csv(f'{folder_path}train_identity.csv')
train_transaction = pd.read_csv(f'{folder_path}train_transaction.csv')
test_identity = pd.read_csv(f'{folder_path}test_identity.csv')
test_transaction = pd.read_csv(f'{folder_path}test_transaction.csv')
submission = pd.read_csv(f'{folder_path}sample_submission.csv')

In [ ]:
print("train_identity shape: ", train_identity.shape)
print("train_transaction shape: ", train_transaction.shape)
print("test_identity shape: ", test_identity.shape)
print("test_transaction shape: ", test_transaction.shape)
print("submission shape: ", submission.shape)

# First glance at the data

In [ ]:
train_identity.head().T

In [ ]:
train_transaction.head().T

In [ ]:
test_identity.head().T

In [ ]:
test_transaction.head().T

# Data quality check

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(30, 20))
matplotlib_venn.venn2(
    subsets=[
        set(train_transaction.TransactionID),
        set(train_identity.TransactionID)
    ],
    set_labels=[
        "Transaction",
        "Identity"
    ],
    ax=ax1
)
matplotlib_venn.venn2(
    subsets=[
        set(test_transaction.TransactionID),
        set(test_identity.TransactionID)
    ],
    set_labels=[
        "Transaction",
        "Identity"
    ],
    ax=ax2
)
ax1.set_title("Train")
ax2.set_title("Test")
plt.suptitle("Intersection of transaction and identity TransactionIDs")
plt.tight_layout()
plt.show()

In [ ]:
train_transaction["has_identity_info"] = train_transaction.TransactionID.isin(
    train_identity.TransactionID
)

test_transaction["has_identity_info"] = test_transaction.TransactionID.isin(
    test_identity.TransactionID
)

In [ ]:
train_transaction_na_flags = train_transaction[[
        column for column in train_transaction.columns.values if column not in [
            "isFraud", "has_identity_info", "TransactionID", "TransactionDT"
        ]
    ]
].isna()

test_transaction_na_flags = test_transaction[[
        column for column in test_transaction.columns.values if column not in [
            "has_identity_info", "TransactionID", "TransactionDT"
        ]
    ]
].isna()

train_transaction_na_flags["isFraud"] = train_transaction["isFraud"]
train_transaction_na_flags_gr = train_transaction_na_flags.groupby("isFraud").mean() * 100

train_transaction_na_flags["has_identity_info"] = train_transaction["has_identity_info"]
train_transaction_na_flags["TransactionDT"] = train_transaction["TransactionDT"]

test_transaction_na_flags["has_identity_info"] = test_transaction["has_identity_info"]
test_transaction_na_flags["TransactionDT"] = test_transaction["TransactionDT"]

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(30, 10))

sns.countplot(
    y="isFraud",
    hue="has_identity_info",
    data=train_transaction,
    ax=ax1
)

sns.boxplot(
    y="isFraud",
    x="value",
    data=pd.melt(train_transaction_na_flags_gr.T),
    ax=ax2,
    orient="h",
    showmeans=True,
    meanline=True,
    meanprops=dict(linestyle='-.', linewidth=2.5, color='red')
)

# https://matplotlib.org/3.1.0/gallery/statistics/bxp.html

ax1.set_title("The number of transactions without identity info")

ax2.set_title("The distribution of null-values percentages for transactions features")

plt.suptitle("An overview of data completeness for train part of the data")

ax1.grid(True)

ax2.grid(True)

plt.show()

In [ ]:
print("Descriptive statistics about null-values in features of transactions:")

print()

print(train_transaction_na_flags_gr.T.describe())

train_transaction_na_flags_gr_t = train_transaction_na_flags_gr.T

print()

print("So what we have is:")

print("\t- 50% of non-fraud train transactions have > 28% of features with null-values;")

print("\t- 50% of fraud train transactions have > 29% of features with null-values;")

print("\t- 25% of non-fraud train transactions have > 78% of features with null-values;")

print("\t- 25% of fraud train transactions have > 50% of features with null-values;")

print(
    "\t- there are {} non-fraud train transactions where > 90% features have null-values;".format(
        train_transaction_na_flags_gr_t[train_transaction_na_flags_gr_t[0] > 90][0].count()
    )
)

print(
    "\t- there are {} fraud train transactions where > 80% features have null-values.".format(
        train_transaction_na_flags_gr_t[train_transaction_na_flags_gr_t[1] > 80][1].count()
    )
)

print("\t- there are some non-fraud train transactions where > 94% features have null-values;")

print("\t- there are some fraud train transactions where > 82% features have null-values.")

print()

print(
    "Percent of non-fraud train transactions with identity info:",
    train_transaction[train_transaction.isFraud == 0].has_identity_info.mean() * 100
)

print()

print(
    "Percent of fraud train transactions with identity info:",
    train_transaction[train_transaction.isFraud == 1].has_identity_info.mean() * 100
)

In [ ]:
train_transaction_na_flags_gr_t_m = pd.melt(
    train_transaction_na_flags_gr.T.reset_index(),
    id_vars=['index'],
    value_vars=[0, 1]
).rename(
    {"index": "features", "value": "percentage"},
    axis=1
)

train_transaction_na_flags_gr_t_m.isFraud = train_transaction_na_flags_gr_t_m.isFraud.astype(np.bool)

plt.figure(figsize=(20, 150))

sns.barplot(
    y="features",
    x="percentage",
    hue="isFraud",
    data=train_transaction_na_flags_gr_t_m,
    order=train_transaction_na_flags_gr.T.sort_values(by=0, ascending=False).index,
    orient='h'
)

plt.title("Percentage of null-values in features for train part of the data")

plt.show()

# Very simple baselines (without any tunning or feature engineering)

In [ ]:
train_transaction_na_flags.sort_values(by="TransactionDT", ascending=True, inplace=True)

In [ ]:
sorted(sklearn.metrics.SCORERS.keys())

In [ ]:
X, y = train_transaction_na_flags[
    [column for column in train_transaction_na_flags.columns.values if column not in ["isFraud", "TransactionDT"]]
], train_transaction_na_flags["isFraud"]

In [ ]:
bernoulli_nb = BernoulliNB()
logistic_regr = LogisticRegression()
nu_svc = NuSVC(probability=True)
c_svc = SVC(probability=True)

In [ ]:
bernoulli_nb_scores = pd.DataFrame(
    cross_validate(
        bernoulli_nb,
        X,
        y,
        cv=StratifiedKFold(30),
        scoring=["roc_auc", "accuracy", "recall", "precision", "f1"],
        n_jobs=4,
        verbose=10
    )
)

In [ ]:
logistic_regr_scores = pd.DataFrame(
    cross_validate(
        logistic_regr,
        X,
        y,
        cv=StratifiedKFold(30),
        scoring=["roc_auc", "accuracy", "recall", "precision", "f1"],
        n_jobs=4,
        verbose=10
    )
)

In [ ]:
import scipy
scipy.test()

In [ ]:
nu_svc_scores = pd.DataFrame(
    cross_validate(
        nu_svc,
        X,
        y,
        cv=StratifiedKFold(30),
        scoring=["roc_auc", "accuracy", "recall", "precision", "f1"],
        n_jobs=4,
        verbose=10
    )
)

In [ ]:
c_svc_scores = pd.DataFrame(
    cross_validate(
        c_svc,
        X,
        y,
        cv=StratifiedKFold(30),
        scoring=["roc_auc", "accuracy", "recall", "precision", "f1"],
        n_jobs=4,
        verbose=10
    )
)

In [ ]:
plt.figure(figsize=(30, 15))
scores[
    [
        "test_roc_auc",
        "test_accuracy",
        "test_recall",
        "test_precision",
        "test_f1"
    ]
].boxplot()
plt.show()

In [ ]:
plt.figure(figsize=(30, 15))
scores[["fit_time", "score_time"]].boxplot()
plt.show()